In [ ]:
pip install symforce

#CODE GENERATION FOR SHOMER TARGET JACOBIANS USING SO(3) X T(3)

In [ ]:
import symforce
import symforce.symbolic as sf
from symforce import codegen
import math

# Symbolic vector and scalar definitions
camera_T = sf.Vector3.symbolic("camera_T")
camera_R = sf.Vector3.symbolic("camera_R")
known_point_G = sf.Vector3.symbolic("known_point_G")

f_x = sf.Symbol('f_x')
f_y = sf.Symbol('f_y')
c_y = sf.Symbol('c_y')
c_x = sf.Symbol('c_x')


def jacobian_matrix(
    camera_T: sf.Vector3,
    camera_R: sf.Vector3,
    known_point_G: sf.Vector3,
    f_x: sf.Scalar,
    f_y: sf.Scalar,
    c_x: sf.Scalar,
    c_y: sf.Scalar
) -> sf.Matrix:

#RODRIGUES ROTATION
    theta = camera_R.norm()

    axis = camera_R / theta
    costheta = sf.cos(theta)
    sintheta = sf.sin(theta)

    #HAT OPERATOR R^3 -> so(3)
    K = sf.Matrix([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ])
    #so(3)->SO(3)
    R = sf.Matrix.eye(3) + sintheta * K + (1 - costheta) * (K * K)

#ROTATE KNOWN GLOBAL POINT
    rotated_point = R * known_point_G
    rotated_point += camera_T

#PINHOLE PROJECT
    xp = (rotated_point[0] / rotated_point[2]) * f_x + c_x
    yp = (rotated_point[1] / rotated_point[2]) * f_y + c_y

#PARTIALS WRT. CAMERA T
    dxp_dT = sf.Vector3(
        sf.diff(xp, camera_T[0]),
        sf.diff(xp, camera_T[1]),
        sf.diff(xp, camera_T[2])
    )
    dyp_dT = sf.Vector3(
        sf.diff(yp, camera_T[0]),
        sf.diff(yp, camera_T[1]),
        sf.diff(yp, camera_T[2])
    )

#PARTIALS WRT. CAMERA R
    dxp_dR = sf.Vector3(
        sf.diff(xp, camera_R[0]),
        sf.diff(xp, camera_R[1]),
        sf.diff(xp, camera_R[2])
    )
    dyp_dR = sf.Vector3(
        sf.diff(yp, camera_R[0]),
        sf.diff(yp, camera_R[1]),
        sf.diff(yp, camera_R[2])
    )

#2X6
    jacobian_2x6 = sf.Matrix([
        [dxp_dT[0], dxp_dT[1], dxp_dT[2], dxp_dR[0], dxp_dR[1], dxp_dR[2]],
        [dyp_dT[0], dyp_dT[1], dyp_dT[2], dyp_dR[0], dyp_dR[1], dyp_dR[2]]
    ])

    return jacobian_2x6


#CODE GEN
jacobian_codegen = codegen.Codegen.function(
    func=jacobian_matrix,
    config=codegen.CppConfig(),
)

#SAVE IT!
jacobian_codegen_data = jacobian_codegen.generate_function()

print("Files generated in {}:\n".format(jacobian_codegen_data.output_dir))
for f in jacobian_codegen_data.generated_files:
    print("  |- {}".format(f.relative_to(jacobian_codegen_data.output_dir)))



    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code, you should set epsilon to either a symbol
    (recommended) or a small numerical value like `sf.numeric_epsilon`.  You should do
    this before importing any other code from symforce, e.g. with

        import symforce
        symforce.set_epsilon_to_symbol()

    or

        import symforce
        symforce.set_epsilon_to_number()

    For more information on use of epsilon to prevent singularities, take a look at the
    Epsilon Tutorial: https://symforce.org/tutorials/epsilon_tutorial.html



Files generated in /tmp/sf_codegen_jacobian_matrix_b9ksl86x:

  |- cpp/symforce/sym/jacobian_matrix.h
